In [1]:
import numpy as np

### Parameters
Assigning parameter values from _Marte  J.  Sætra,  Gaute  T.  Einevoll,  and  Geir  Halnes.   An  electrodiffusive,  ion  conserving  pinsky-rinzelmodel with homeostatic mechanisms.PLOS Computational Biology, 16(4):1–36, 04 2020._

In [9]:
#surface-to-volume ratio, cm^-1
s = (616*10**-12)/(100*1437*10**-18)
#intracellular-to-extracellular volume
r = 2
#membrane specific capacitance, F/cm^2
c = 3*10**-2/(10**4)
#intracellular tortuosity
lambda_in = 3.2
#extracellular tortuosity
lambda_ex = 1.6
#charge of electron, C
e = 1.60217662*10**-19
#Boltzmann's constant, cm^2.kg.s^-2.K^-1
k_B = 1.38064852*10**-23*10**4
#absolute temperature, K
T = 310.15
#Avogadro's constant, mol^-1
N_A = 6.02214076*10**23
#fraction of free calcium concentration
gamma = 0.01
#diffusion coefficient, cm^2/s
D_Ca = 0.71*10**-9*10**4
D_Na = 1.33*10**-9*10**4
D_K = 1.96*10**-9*10**4
D_Cl = 2.03*10**-9*10**4
#initial concentration of intracellular ions, mol/cm^3
Ca_i0 = 0.01/(1000*1000)
Na_i0 = 15/(1000*1000)
K_i0 = 140/(1000*1000)
Cl_i0 = 4/(1000*1000)
X_i0 = 151/(1000*1000)
#initial concentration of extracellular ions, mol/cm^3
Ca_e0 = 1.1/(1000*1000)
Na_e0 = 145/(1000*1000)
K_e0 = 5/(1000*1000)
Cl_e0 = 110/(1000*1000)
X_e0 = 42.2/(1000*1000)
#diffusion coefficient of nonspecific ion, averaged and normalized by total initial concentration, cm^2/S
D_w = (D_Na*Na_i0+D_K*K_i0+D_Cl*Cl_i0)/(Na_i0+K_i0+Cl_i0+X_i0)
D_y = (D_Ca*Ca_e0+D_Na*Na_e0+D_K*K_e0+D_Cl*Cl_e0)/(Ca_e0+Na_e0+K_e0+Cl_e0+X_e0)
#free charge concentration
u_f = N_A*gamma*Ca_i0
w_f = N_A*(Na_i0+K_i0+Cl_i0)
y_f = N_A*(Ca_e0+Na_e0+K_e0+Cl_e0)
#initial charge densities
u_0 = 2*e*N_A*gamma*Ca_i0
w_0 = e*N_A*(Na_i0+K_i0-Cl_i0-X_i0)
y_0 = e*N_A*(2*Ca_e0+Na_e0+K_e0-Cl_e0-X_e0)

The ionic diffusion coefficient ($cm^2\cdot s^{-1}$) and the conductivity ($S\cdot cm^{-1}$) of intracellular calcium and nonspecific ions on both side of the membrane are estimated as

$$a_n = \frac{\gamma_n D_n}{\lambda^2}$$
and
$$\sigma_n = \frac{q_e^2n_f}{k_BT}$$
respectively.

In [15]:
a_u = gamma*D_Ca/lambda_in**2
a_w = D_w/lambda_in**2
a_y = D_y/lambda_ex**2

sigma_u = (e**2*u_f)/(k_B*T)
sigma_w = (e**2*w_f)/(k_B*T)
sigma_y = (e**2*y_f)/(k_B*T)
sigma_in = sigma_u + sigma_w

np.array([a_u,a_w,a_y,sigma_u,sigma_w,sigma_y])

array([6.93359375e-09, 9.52841482e-07, 5.49593791e-06, 3.61007949e-06,
       5.74002639e-02, 9.42591756e-02])

Define the diffusion coefficient matrix:

$$\mathbf{A}=
    \begin{pmatrix}
        a_y+\frac{\sigma_y(\sigma_{in}-s a_y c)}{s c(r\sigma_{in}+\sigma_y)} & \frac{\sigma_y a_u}{s c(r\sigma_{in}+\sigma_y)} & \frac{\sigma_y a_w}{s c(r\sigma_{in}+\sigma_y)}  \\
         & & \\
        \frac{\sigma_u(\sigma_y+rs a_yc)}{r\sigma_{in}+\sigma_y}    & a_u - \frac{\sigma_ur a_w}{r\sigma_{in}+\sigma_y} & \frac{\sigma_ur a_u}{r\sigma_{in}+\sigma_y}  \\
         & & \\
        \frac{\sigma_w(\sigma_y+rs a_yc)}{r\sigma_{in}+\sigma_y} & \frac{\sigma_wr a_u}{r\sigma_{in}+\sigma_y} & a_w-\frac{\sigma_wr a_w}{r\sigma_{in}+\sigma_y}\\
    \end{pmatrix}\in \mathbb{R}_{>0}^{3\times 3},$$

In [26]:
avv = a_y + (sigma_y*(sigma_in-s*a_y*c))/(s*c*(r*sigma_in+sigma_y))
avu = (sigma_y*a_u)/(s*c*(r*sigma_in+sigma_y))
avw = (sigma_y*a_w)/(s*c*(r*sigma_in+sigma_y))

auv = sigma_u*(sigma_y+r*s*a_y*c)/(r*sigma_in+sigma_y)
auu = a_u - sigma_u*r*a_w/(r*sigma_in+sigma_y)
auw = sigma_u*r*a_u/(r*sigma_in+sigma_y)

awv = sigma_w*(sigma_y+r*s*a_y*c)/(r*sigma_in+sigma_y)
awu = sigma_w*r*a_u/(r*sigma_in+sigma_y)
aww = a_w - sigma_w*r*a_w/(r*sigma_in+sigma_y)

A = np.matrix([[avv,avu,avw],[auv,auu,auw],[awv,awu,aww]])
A

matrix([[2.01249571e+00, 2.43081273e-07, 3.34051761e-05],
        [1.62763014e-06, 6.90068722e-09, 2.39452747e-13],
        [2.58793193e-02, 3.80729868e-09, 4.29627655e-07]])

To check that $A$ is a positive definite matrix we recall that a non-symmetric real matrix $A\in M_3(\mathbb{R})$ is positive definite iff all eigenvalues of $A+A^\top$ are positive real numbers.

In [29]:
def is_pos_def(A):
    B = A + A.transpose()
    return np.all(np.linalg.eigvals(B) > 0)
is_pos_def(A)

False